In [ ]:
!cp -r /kaggle/input/efficientnetpytorch /kaggle/working

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import torchvision.models as models
import torch.utils.data as data
import sklearn.utils as utils
from PIL import Image
import torchvision.transforms as T
import sklearn.metrics as metrics
import sys
from efficientnetpytorch.efficientnet_pytorch.model import EfficientNet

In [ ]:
input_path = '/kaggle/input/ranzcr-clip-catheter-line-classification'

In [ ]:
num_classes = 11
device = 'cuda:0'

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.base = EfficientNet.from_name('efficientnet-b7')
        self.dense = nn.Linear(1000, num_classes)
    def forward(self, x):
        x = torch.cat([x, x, x], dim=1)
        return self.dense(self.base(x))

In [ ]:
model = Model().to(device)

In [ ]:
effnet_path = '../input/efficientnetpytorch'
import sys
sys.path.append(effnet_path)

In [ ]:
# model.load_state_dict(torch.load(f'/kaggle/input/efficientnetpytorch/efficientnet-model/model_10.pth'))

In [ ]:
df = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/sample_submission.csv')

In [ ]:
class TestDataset(data.Dataset):
    def __init__(self, df, transforms):
        super(TestDataset, self).__init__()
        self.df = df
        self.transforms = transforms
    def __getitem__(self, index):
        row = self.df.iloc[index]
        img = Image.open(f'{input_path}/test/{row[0]}.jpg')
        return self.transforms(img)
    def __len__(self):
        return len(self.df)

In [ ]:
test_transforms = T.Compose([
    T.Resize((512, 512)),
    T.ToTensor(),
    T.Normalize((0.5,), (0.5,))
])

In [ ]:
test_dataset = TestDataset(df=df, transforms=test_transforms)
test_dataloader = data.DataLoader(dataset=test_dataset, num_workers=2, shuffle=False, batch_size=4)

In [ ]:
len(test_dataloader)

In [ ]:
model.eval()
y_preds = []
for b, x in enumerate(test_dataloader):
        if b%100==0 or b==len(test_dataloader):
            print(b)
        x = x.cuda()
        y_pred = model(x).detach()
        y_preds += y_pred.cpu().numpy().tolist()

In [ ]:
def sigmoid(x):
    x = x.astype('float32')
    x[x < -709.] = -709.
    return 1. / (1. + np.exp(-x))

In [ ]:
y_preds = sigmoid(np.array(y_preds))

In [ ]:
y_preds.shape

In [ ]:
columns = df.columns

In [ ]:
df[columns[1:]].shape

In [ ]:
df

In [ ]:
df[columns[1:]] = y_preds

In [ ]:
df

In [ ]:
df.to_csv('submission.csv', index=False)